In [1]:
import mlrun

In [2]:
source = 'git://github.com/GiladShapira94/load-project-example.git#master'

By Specify a url MLRun automatically clone your git repository

if Clone = True, always clone (delete any existing content)

In [3]:
project = mlrun.load_project('./clone',url=source,init_git=True,clone=True)

In [4]:
print(project.to_yaml())

kind: project
metadata:
  name: from-git
spec:
  functions:
  - name: nuclio
    kind: nuclio
    image: mlrun/mlrun
    handler: multi:multi_3
    with_repo: true
  - name: my_job
    kind: job
    image: mlrun/mlrun
    handler: my_job.func
    with_repo: true
  - url: serving.yaml
    name: serving
    with_repo: true
  workflows: []
  artifacts: []
  source: git://github.com/GiladShapira94/load-project-example.git#refs/heads/master
  origin_url: git://github.com/GiladShapira94/load-project-example.git#refs/heads/master
  load_source_on_run: true
  desired_state: online



In [5]:
project.run_function(function='my_job',params={'num':3},local=True)

> 2022-08-03 07:33:54,551 [warning] run command, file or code were not specified
> 2022-08-03 07:33:54,553 [info] starting run my_job-func uid=7429a1aaa85e4e3cbbd212fbab88a710 DB=http://mlrun-api:8080
24


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
from-git,...ab88a710,0,Aug 03 07:33:54,completed,my_job-func,v3io_user=shapirakind=localowner=shapirahost=jupyter-shapira-6d47768cf9-42z4t,,num=3,num=24return=24,


> 2022-08-03 07:33:55,028 [info] run executed, status=completed


In [6]:
project.run_function(function='my_job',params={'num':3},local=False)

> 2022-08-03 07:33:55,034 [info] starting run my_job-func uid=4a5bb09f0b24432a91287d7ec7586388 DB=http://mlrun-api:8080
> 2022-08-03 07:33:55,259 [info] Job is running in the background, pod: my-job-func-qct84
> 2022-08-03 07:33:59,290 [info] extracting source from git://github.com/GiladShapira94/load-project-example.git#refs/heads/master to /mlrun/code
24
> 2022-08-03 07:33:59,450 [info] run executed, status=completed
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
from-git,...c7586388,0,Aug 03 07:33:59,completed,my_job-func,v3io_user=shapirakind=jobowner=shapiramlrun/client_version=1.1.0-rc12host=my-job-func-qct84,,num=3,num=24return=24,


> 2022-08-03 07:34:01,751 [info] run executed, status=completed


In [8]:
serving_func = project.get_function('serving')

In [9]:
serving_server = serving_func.to_mock_server()

> 2022-08-03 07:35:55,089 [warning] run command, file or code were not specified


In [10]:
serving_server.test('/',5)

Start
End


13

In [11]:
project.set_source(source,pull_at_runtime=False)
print(project.to_yaml())

kind: project
metadata:
  name: from-git
spec:
  functions:
  - name: nuclio
    kind: nuclio
    image: mlrun/mlrun
    handler: multi:multi_3
    with_repo: true
  - name: my_job
    kind: job
    image: mlrun/mlrun
    handler: my_job.func
    with_repo: true
  - url: serving.yaml
    name: serving
    with_repo: true
  workflows: []
  artifacts: []
  source: git://github.com/GiladShapira94/load-project-example.git#master
  origin_url: git://github.com/GiladShapira94/load-project-example.git#refs/heads/master
  load_source_on_run: false
  desired_state: online



In [12]:
serving_dep = project.deploy_function('serving')

> 2022-08-03 07:36:00,372 [info] Starting remote function deploy
2022-08-03 07:36:01  (info) Deploying function
2022-08-03 07:36:01  (info) Building
2022-08-03 07:36:01  (info) Staging files and preparing base images
2022-08-03 07:36:02  (info) Building processor image
2022-08-03 07:37:27  (info) Build complete
2022-08-03 07:37:35  (info) Function deploy complete
> 2022-08-03 07:37:35,523 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-from-git-serving.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['from-git-serving-from-git.default-tenant.app.cust-cs-il-3-4-1.iguazio-cd1.com/']}


In [13]:
serving_dep.function.invoke('/',5)

> 2022-08-03 07:37:35,611 [info] invoking function: {'method': 'POST', 'path': 'http://nuclio-from-git-serving.default-tenant.svc.cluster.local:8080/'}


13

In [14]:
nuclio_func = project.deploy_function('nuclio')

> 2022-08-03 07:37:35,647 [info] Starting remote function deploy
2022-08-03 07:37:35  (info) Deploying function
2022-08-03 07:37:35  (info) Building
2022-08-03 07:37:36  (info) Staging files and preparing base images
2022-08-03 07:37:36  (info) Building processor image
2022-08-03 07:38:51  (info) Build complete
2022-08-03 07:38:57  (info) Function deploy complete
> 2022-08-03 07:38:58,112 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-from-git-nuclio.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['from-git-nuclio-from-git.default-tenant.app.cust-cs-il-3-4-1.iguazio-cd1.com/']}


In [15]:
nuclio_func.function.invoke('/',{'int':4})

> 2022-08-03 07:38:58,194 [info] invoking function: {'method': 'POST', 'path': 'http://nuclio-from-git-nuclio.default-tenant.svc.cluster.local:8080/'}


{'int': 12}